# CHOMP SOLVER

In [1]:
# This is our "HashMap" for memoization.
# It will store {state_tuple: is_win (bool)}
win_cache = {}

def get_next_state(current_state: tuple, r: int, c: int) -> tuple:
    """
    Calculates the new state tuple after a move at (r, c).
    """
    # Convert to list for modification
    next_state_list = list(current_state)
    
    # A move at (r, c) truncates all rows >= r at column c
    for i in range(r, len(next_state_list)):
        next_state_list[i] = min(next_state_list[i], c)
        
    # --- State Normalization ---
    # To use our cache effectively, we must normalize the state.
    # (4, 2, 2) is the same as (4, 2, 2, 0, 0).
    # We remove trailing zeros, as they don't represent valid squares.
    while next_state_list and next_state_list[-1] == 0:
        next_state_list.pop()
        
    return tuple(next_state_list)

def is_winning_state(state: tuple) -> bool:
    """
    Recursively determines if a state is a winning (N) or losing (P)
    position using our cache (dict).
    Returns True if 'state' is a winning position, False if losing.
    """
    # 1. Check the "HashMap" (cache) first
    if state in win_cache:
        return win_cache[state]

    # 2. Base Case: The empty board ().
    # A player faced with this state has no moves and LOSES.
    if state == ():
        win_cache[state] = False
        return False
        
    # 3. Iterate through all possible moves from this state
    
    # We assume this is a losing state until we find a winning move
    can_move_to_losing_state = False
    
    # Iterate through each row 'r'
    for r in range(len(state)):
        # Iterate through each column 'c' in that row
        for c in range(state[r]):
            
            # Check for the losing move
            if r == 0 and c == 0:
                # If this is the *only* possible move (i.e., state is (1,)),
                # this loop will finish, can_move_to_losing_state will be False,
                # and the function will correctly return False.
                # Otherwise, we just skip this move.
                continue

            # This is a valid, non-losing move.
            # Get the resulting state from this move.
            next_state = get_next_state(state, r, c)
            
            # Recursively check if the *opponent* LOSES from that next_state.
            if not is_winning_state(next_state):
                # We found a move to a losing position for the opponent.
                # This means the *current* state is a WINNING position.
                can_move_to_losing_state = True
                break
        
        if can_move_to_losing_state:
            break
            
    # 4. Cache and Return Result
    #
    # If we found a move to a losing state, this is a winning state.
    # If we looped through all moves and *none* led to a losing state
    # (or if the only move was (0,0)), this is a losing state.
    win_cache[state] = can_move_to_losing_state
    return can_move_to_losing_state

## THIRD ROW VALUE ANALYSIS

- If r = 1, the only P-positions are (3,1,1) and (2,2,1).
- If r = 2, the P-positions are (a+4, a+2, 2) for nonnegative a.
- If r = 3, the P-positions are (6,3,3), (7,4,3) and (5,5,3).
- If r = 4, the P-positions are (8,4,4), (9,5,4), (10,6,4) and (7,7,4).
- If r = 5, the P-positions are (10,5,5), (9,6,5) and (a+11,a+7,5) for nonnegative a.
- If r = 6, the P-positions are (9, 9, 6), (11, 6, 6), (12, 7, 6) and (13, 8, 6).
- If r = 7, the P-positions are (12, 9, 7), (13, 7, 7), (14, 8, 7) and (a+15, a+10, 7) for nonnegative a.
- If r = 8, the P-positions are (12, 12, 8), (14, 9, 8), (15, 8, 8), (16, 10, 8) and (17, 11, 8).
- If r = 9, the P-positions are (14, 11, 9), (16, 9, 9), (17, 10, 9) and (a+18, a+23, 9) for nonnegative a.
- If r = 10, the P-positions are (14, 14, 10), (18, 10, 10), (19, 11, 10), (20, 12, 10) and (21, 13, 10)
- If r = 11, the P-positions are (17, 14, 11), (19, 12, 11), (20, 11, 11), (22, 13, 11), (21, 18, 11) and (a+23, a+15, 11)

### With any r_3, whenever there're P-positions of both forms (r_1, r_1, r_3) and (r_1, r_3, r_3), and there's no P-Position in the form (r_1, r_1, r_3+1) there's a linear robust family in r_3 + 1

### If there's a robust family with some r_3, there're P-positions of both forms (r_1, r_1, r_3-1) and (r_1, r_3-1, r_3-1).

### The gap is ever growing between robust families --> prove that if an r_3 has a linear family, r_3 + 1 can't have one

In [9]:
for i in range(5, 101):
    board = (i, i, i)
    is_win = is_winning_state(board)
    # print(f"A 3x{i} board {board} is a {'WIN' if is_win else 'LOSS'} for the first player.")

for i, board in enumerate(win_cache.items()):
    if board[1] == False and len(board[0]) == 3 and board[0][2] == 16:
        # print(f"State: {board[0]}, is a LOSS position.")
        print(board[0])

(23, 23, 16)
(26, 18, 16)
(28, 16, 16)
(29, 17, 16)
(31, 19, 16)
(30, 20, 16)
(32, 21, 16)
(33, 22, 16)


## (a+4, a+2, 2)

In [4]:
win_cache = {}
for i in range(5, 21):
    board = (i, i, i)
    is_win = is_winning_state(board)
    # print(f"A 3x{i} board {board} is a {'WIN' if is_win else 'LOSS'} for the first player.")

for i, board in enumerate(win_cache.items()):
    if board[1] == False and len(board[0]) == 3 and board[0][2] == 2:
        print(f"State: {board[0]}, is a LOSS position.")

State: (4, 2, 2), is a LOSS position.
State: (5, 3, 2), is a LOSS position.
State: (6, 4, 2), is a LOSS position.
State: (7, 5, 2), is a LOSS position.
State: (8, 6, 2), is a LOSS position.
State: (9, 7, 2), is a LOSS position.
State: (10, 8, 2), is a LOSS position.
State: (11, 9, 2), is a LOSS position.
State: (12, 10, 2), is a LOSS position.
State: (13, 11, 2), is a LOSS position.
State: (14, 12, 2), is a LOSS position.
State: (15, 13, 2), is a LOSS position.
State: (16, 14, 2), is a LOSS position.
State: (17, 15, 2), is a LOSS position.
State: (18, 16, 2), is a LOSS position.
State: (19, 17, 2), is a LOSS position.


- All states (i, i-2, 2) where i >= 5 is a P-position.

- A 3x5 board (5, 3, 2) is a LOSS for the first player.
- A 3x6 board (6, 4, 2) is a LOSS for the first player.
- A 3x7 board (7, 5, 2) is a LOSS for the first player.
- A 3x8 board (8, 6, 2) is a LOSS for the first player.
- A 3x9 board (9, 7, 2) is a LOSS for the first player.
- A 3x10 board (10, 8, 2) is a LOSS for the first player.
- A 3x11 board (11, 9, 2) is a LOSS for the first player.
- A 3x12 board (12, 10, 2) is a LOSS for the first player.
- A 3x13 board (13, 11, 2) is a LOSS for the first player.
- A 3x14 board (14, 12, 2) is a LOSS for the first player.
- A 3x15 board (15, 13, 2) is a LOSS for the first player.
- A 3x16 board (16, 14, 2) is a LOSS for the first player.
- A 3x17 board (17, 15, 2) is a LOSS for the first player.
- A 3x18 board (18, 16, 2) is a LOSS for the first player.
- A 3x19 board (19, 17, 2) is a LOSS for the first player.
- A 3x20 board (20, 18, 2) is a LOSS for the first player.
- A 3x21 board (21, 19, 2) is a LOSS for the first player.
- A 3x22 board (22, 20, 2) is a LOSS for the first player.
- A 3x23 board (23, 21, 2) is a LOSS for the first player.
- A 3x24 board (24, 22, 2) is a LOSS for the first player.
- A 3x25 board (25, 23, 2) is a LOSS for the first player.
- A 3x26 board (26, 24, 2) is a LOSS for the first player.
- A 3x27 board (27, 25, 2) is a LOSS for the first player.
- A 3x28 board (28, 26, 2) is a LOSS for the first player.
- A 3x29 board (29, 27, 2) is a LOSS for the first player.
- A 3x30 board (30, 28, 2) is a LOSS for the first player.
- A 3x31 board (31, 29, 2) is a LOSS for the first player.
- A 3x32 board (32, 30, 2) is a LOSS for the first player.
- A 3x33 board (33, 31, 2) is a LOSS for the first player.
- A 3x34 board (34, 32, 2) is a LOSS for the first player.
- A 3x35 board (35, 33, 2) is a LOSS for the first player.

## FIRST MOVE ANALYSYS

In [19]:
win_cache = {}
for i in range(1, 161):
    board = (i, i, i)
    is_win = is_winning_state(board)
    # print(f"A 3x{i} board {board} is a {'WIN' if is_win else 'LOSS'} for the first player.")

In [20]:
first_two = []
last_two = []
for i, board in enumerate(win_cache.items()):

    if board[1] == False:
        if len(board[0]) >= 2 and board[0][0] == board[0][1]:
            print(f"State: {board[0]}, is a LOSS position (first two match).")
            first_two.append(board[0][0])
            
        elif len(board[0]) >= 3 and board[0][1] == board[0][2]:
            print(f"State: {board[0]}, is a LOSS position (second/third match).")
            last_two.append(board[0][0])
print(first_two)
print(last_two)

State: (2, 2, 1), is a LOSS position (first two match).
State: (3, 1, 1), is a LOSS position (second/third match).
State: (4, 2, 2), is a LOSS position (second/third match).
State: (5, 5, 3), is a LOSS position (first two match).
State: (6, 3, 3), is a LOSS position (second/third match).
State: (7, 7, 4), is a LOSS position (first two match).
State: (8, 4, 4), is a LOSS position (second/third match).
State: (9, 9, 6), is a LOSS position (first two match).
State: (10, 5, 5), is a LOSS position (second/third match).
State: (11, 6, 6), is a LOSS position (second/third match).
State: (12, 12, 8), is a LOSS position (first two match).
State: (13, 7, 7), is a LOSS position (second/third match).
State: (14, 14, 10), is a LOSS position (first two match).
State: (15, 8, 8), is a LOSS position (second/third match).
State: (16, 9, 9), is a LOSS position (second/third match).
State: (17, 17, 12), is a LOSS position (first two match).
State: (18, 10, 10), is a LOSS position (second/third match).
St

## FIRST MOVE: (Put the difference between first and third row for the first set through oeis)
- If third row has a certain number of blocks, & the first & second are above a certain floor, it's a winning position.
Next move: (5,5,3), (7,7,4), (9,9,6), (17,17,12), (19,19,13), etc.

[2, 5, 7, 9, 12, 14, 17, 19, 22, 23, 26, 29, 31, 33, 36, 38, 41, 43, 46, 47, 50, 52, 55, 58, 61, 62, 65, 67, 69, 71, 75, 77, 79, 81, 84, 86, 90, 91, 94, 97, 99, 101, 103, 106, 108, 110, 114, 115, 119, 120, 123, 126, 127, 131, 132, 135, 137, 140, 142, 145, 147, 149, 152, 154, 157, 160]


- If the first row has a certain number of blocks & the second + third is above a certain floor, it's a winning position 
Next move: (3,1,1), (11,6,6), (13,7,7), (15,8,8), (21,12,12), etc. Each of below's winning move is (n, i, i) (i corresponds to the index n is found in)

[3, 4, 6, 8, 10, 11, 13, 15, 16, 18, 20, 21, 24, 25, 27, 28, 30, 32, 34, 35, 37, 39, 40, 42, 44, 45, 48, 49, 51, 53, 54, 56, 57, 59, 60, 63, 64, 66, 68, 70, 72, 73, 74, 76, 78, 80, 82, 83, 85, 87, 88, 89, 92, 93, 95, 96, 98, 100, 102, 104, 105, 107, 109, 111, 112, 113, 116, 117, 118, 121, 122, 124, 125, 128, 129, 130, 133, 134, 136, 138, 139, 141, 143, 144, 146, 148, 150, 151, 153, 155, 156, 158, 159]



In [15]:
winning_moves = set(first_two) | set(last_two)
# winning_moves = sorted(list(winning_moves))
print(winning_moves)
for i in range(2, 161):
    if i not in winning_moves:
        print("missing: ", i)

{2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160}


State: (2, 2, 1), is a LOSS position (first two match).
State: (3, 1, 1), is a LOSS position (second/third match).
State: (4, 2, 2), is a LOSS position (second/third match).
State: (5, 5, 3), is a LOSS position (first two match).
State: (6, 3, 3), is a LOSS position (second/third match).
State: (7, 7, 4), is a LOSS position (first two match).
State: (8, 4, 4), is a LOSS position (second/third match).
State: (9, 9, 6), is a LOSS position (first two match).
State: (10, 5, 5), is a LOSS position (second/third match).
State: (11, 6, 6), is a LOSS position (second/third match).
State: (12, 12, 8), is a LOSS position (first two match).
State: (13, 7, 7), is a LOSS position (second/third match).
State: (14, 14, 10), is a LOSS position (first two match).
State: (15, 8, 8), is a LOSS position (second/third match).
State: (16, 9, 9), is a LOSS position (second/third match).
State: (17, 17, 12), is a LOSS position (first two match).
State: (18, 10, 10), is a LOSS position (second/third match).
State: (19, 19, 13), is a LOSS position (first two match).
State: (20, 11, 11), is a LOSS position (second/third match).
State: (21, 12, 12), is a LOSS position (second/third match).
State: (22, 22, 15), is a LOSS position (first two match).
State: (23, 23, 16), is a LOSS position (first two match).
State: (24, 13, 13), is a LOSS position (second/third match).
State: (25, 14, 14), is a LOSS position (second/third match).
State: (26, 26, 18), is a LOSS position (first two match).
State: (27, 15, 15), is a LOSS position (second/third match).
State: (28, 16, 16), is a LOSS position (second/third match).
State: (29, 29, 20), is a LOSS position (first two match).
State: (30, 17, 17), is a LOSS position (second/third match).
State: (31, 31, 21), is a LOSS position (first two match).
State: (32, 18, 18), is a LOSS position (second/third match).
State: (33, 33, 23), is a LOSS position (first two match).
State: (34, 19, 19), is a LOSS position (second/third match).
State: (35, 20, 20), is a LOSS position (second/third match).
State: (36, 36, 25), is a LOSS position (first two match).
State: (37, 21, 21), is a LOSS position (second/third match).
State: (38, 38, 27), is a LOSS position (first two match).
State: (39, 22, 22), is a LOSS position (second/third match).
State: (40, 23, 23), is a LOSS position (second/third match).
State: (41, 41, 29), is a LOSS position (first two match).
State: (42, 24, 24), is a LOSS position (second/third match).
State: (43, 43, 30), is a LOSS position (first two match).
State: (44, 25, 25), is a LOSS position (second/third match).
State: (45, 26, 26), is a LOSS position (second/third match).
State: (46, 46, 32), is a LOSS position (first two match).
State: (47, 47, 33), is a LOSS position (first two match).
State: (48, 27, 27), is a LOSS position (second/third match).
State: (49, 28, 28), is a LOSS position (second/third match).
State: (50, 50, 35), is a LOSS position (first two match).
State: (51, 29, 29), is a LOSS position (second/third match).
State: (52, 52, 37), is a LOSS position (first two match).
State: (53, 30, 30), is a LOSS position (second/third match).
State: (54, 31, 31), is a LOSS position (second/third match).
State: (55, 55, 39), is a LOSS position (first two match).
State: (56, 32, 32), is a LOSS position (second/third match).
State: (57, 33, 33), is a LOSS position (second/third match).
State: (58, 58, 41), is a LOSS position (first two match).
State: (59, 34, 34), is a LOSS position (second/third match).
State: (60, 35, 35), is a LOSS position (second/third match).
State: (61, 61, 43), is a LOSS position (first two match).
State: (62, 62, 44), is a LOSS position (first two match).
State: (63, 36, 36), is a LOSS position (second/third match).
State: (64, 37, 37), is a LOSS position (second/third match).
State: (65, 65, 46), is a LOSS position (first two match).
State: (66, 38, 38), is a LOSS position (second/third match).
State: (67, 67, 47), is a LOSS position (first two match).
State: (68, 39, 39), is a LOSS position (second/third match).
State: (69, 69, 49), is a LOSS position (first two match).
State: (70, 40, 40), is a LOSS position (second/third match).
State: (71, 71, 50), is a LOSS position (first two match).
State: (72, 41, 41), is a LOSS position (second/third match).
State: (73, 42, 42), is a LOSS position (second/third match).
State: (74, 43, 43), is a LOSS position (second/third match).
State: (75, 75, 52), is a LOSS position (first two match).
State: (76, 44, 44), is a LOSS position (second/third match).
State: (77, 77, 54), is a LOSS position (first two match).
State: (78, 45, 45), is a LOSS position (second/third match).
State: (79, 79, 55), is a LOSS position (first two match).
State: (80, 46, 46), is a LOSS position (second/third match).
State: (81, 81, 57), is a LOSS position (first two match).
State: (82, 47, 47), is a LOSS position (second/third match).
State: (83, 48, 48), is a LOSS position (second/third match).
State: (84, 84, 59), is a LOSS position (first two match).
State: (85, 49, 49), is a LOSS position (second/third match).
State: (86, 86, 61), is a LOSS position (first two match).
State: (87, 50, 50), is a LOSS position (second/third match).
State: (88, 52, 52), is a LOSS position (second/third match).
State: (89, 51, 51), is a LOSS position (second/third match).
State: (90, 90, 63), is a LOSS position (first two match).
State: (91, 91, 64), is a LOSS position (first two match).
State: (92, 53, 53), is a LOSS position (second/third match).
State: (93, 54, 54), is a LOSS position (second/third match).
State: (94, 94, 66), is a LOSS position (first two match).
State: (95, 55, 55), is a LOSS position (second/third match).
State: (96, 56, 56), is a LOSS position (second/third match).
State: (97, 97, 68), is a LOSS position (first two match).
State: (98, 57, 57), is a LOSS position (second/third match).
State: (99, 99, 70), is a LOSS position (first two match).
State: (100, 58, 58), is a LOSS position (second/third match).
State: (101, 101, 71), is a LOSS position (first two match).
State: (102, 59, 59), is a LOSS position (second/third match).
State: (103, 103, 73), is a LOSS position (first two match).
State: (104, 60, 60), is a LOSS position (second/third match).
State: (105, 61, 61), is a LOSS position (second/third match).
State: (106, 106, 75), is a LOSS position (first two match).
State: (107, 62, 62), is a LOSS position (second/third match).
State: (108, 108, 76), is a LOSS position (first two match).
State: (109, 63, 63), is a LOSS position (second/third match).
State: (110, 110, 78), is a LOSS position (first two match).
State: (111, 64, 64), is a LOSS position (second/third match).
State: (112, 65, 65), is a LOSS position (second/third match).
State: (113, 66, 66), is a LOSS position (second/third match).
State: (114, 114, 80), is a LOSS position (first two match).
State: (115, 115, 81), is a LOSS position (first two match).
State: (116, 67, 67), is a LOSS position (second/third match).
State: (117, 68, 68), is a LOSS position (second/third match).
State: (118, 69, 69), is a LOSS position (second/third match).
State: (119, 119, 83), is a LOSS position (first two match).
State: (120, 120, 85), is a LOSS position (first two match).
State: (121, 70, 70), is a LOSS position (second/third match).
State: (122, 71, 71), is a LOSS position (second/third match).
State: (123, 123, 87), is a LOSS position (first two match).
State: (124, 72, 72), is a LOSS position (second/third match).
State: (125, 73, 73), is a LOSS position (second/third match).
State: (126, 126, 89), is a LOSS position (first two match).
State: (127, 127, 90), is a LOSS position (first two match).
State: (128, 74, 74), is a LOSS position (second/third match).
State: (129, 75, 75), is a LOSS position (second/third match).
State: (130, 76, 76), is a LOSS position (second/third match).
State: (131, 131, 92), is a LOSS position (first two match).
State: (132, 132, 93), is a LOSS position (first two match).
State: (133, 77, 77), is a LOSS position (second/third match).
State: (134, 78, 78), is a LOSS position (second/third match).
State: (135, 135, 95), is a LOSS position (first two match).
State: (136, 79, 79), is a LOSS position (second/third match).
State: (137, 137, 96), is a LOSS position (first two match).
State: (138, 80, 80), is a LOSS position (second/third match).
State: (139, 81, 81), is a LOSS position (second/third match).
State: (140, 140, 98), is a LOSS position (first two match).
State: (141, 82, 82), is a LOSS position (second/third match).
State: (142, 142, 100), is a LOSS position (first two match).
State: (143, 83, 83), is a LOSS position (second/third match).
State: (144, 84, 84), is a LOSS position (second/third match).
State: (145, 145, 102), is a LOSS position (first two match).
State: (146, 85, 85), is a LOSS position (second/third match).
State: (147, 147, 103), is a LOSS position (first two match).
State: (148, 86, 86), is a LOSS position (second/third match).
State: (149, 149, 105), is a LOSS position (first two match).
State: (150, 87, 87), is a LOSS position (second/third match).
State: (151, 88, 88), is a LOSS position (second/third match).
State: (152, 152, 107), is a LOSS position (first two match).
State: (153, 89, 89), is a LOSS position (second/third match).
State: (154, 154, 109), is a LOSS position (first two match).
State: (155, 90, 90), is a LOSS position (second/third match).
State: (156, 91, 91), is a LOSS position (second/third match).
State: (157, 157, 111), is a LOSS position (first two match).
State: (158, 92, 92), is a LOSS position (second/third match).
State: (159, 93, 93), is a LOSS position (second/third match).
State: (160, 160, 113), is a LOSS position (first two match).
[2, 5, 7, 9, 12, 14, 17, 19, 22, 23, 26, 29, 31, 33, 36, 38, 41, 43, 46, 47, 50, 52, 55, 58, 61, 62, 65, 67, 69, 71, 75, 77, 79, 81, 84, 86, 90, 91, 94, 97, 99, 101, 103, 106, 108, 110, 114, 115, 119, 120, 123, 126, 127, 131, 132, 135, 137, 140, 142, 145, 147, 149, 152, 154, 157, 160]
[3, 4, 6, 8, 10, 11, 13, 15, 16, 18, 20, 21, 24, 25, 27, 28, 30, 32, 34, 35, 37, 39, 40, 42, 44, 45, 48, 49, 51, 53, 54, 56, 57, 59, 60, 63, 64, 66, 68, 70, 72, 73, 74, 76, 78, 80, 82, 83, 85, 87, 88, 89, 92, 93, 95, 96, 98, 100, 102, 104, 105, 107, 109, 111, 112, 113, 116, 117, 118, 121, 122, 124, 125, 128, 129, 130, 133, 134, 136, 138, 139, 141, 143, 144, 146, 148, 150, 151, 153, 155, 156, 158, 159]

## SPRAGUE-GRUNDY ANALYSIS

In [ ]:
from functools import lru_cache

def mex(s):
    """Calculates the Minimum Excluded value from a set of non-negative integers."""
    i = 0
    while i in s:
        i += 1
    return i

@lru_cache(maxsize=None)
def sprague_grundy(state):
    """
    Calculates the Sprague-Grundy value (nim-value) for a Chomp state.
    
    Args:
        state (tuple): A tuple of integers representing row lengths. 
                       e.g., (3, 2, 1)
                       Must satisfy r1 >= r2 >= ... >= rk
    
    Returns:
        int: The Grundy value. 0 implies a LOSS position. >0 implies WIN.
    """
    # Base Case: If the state is (1,), only the poisoned piece remains.
    # The player whose turn it is must eat it and loses.
    # Therefore, this is a LOSS position (Value 0).
    if state == (1,):
        return 0
    
    reachable_grundy_values = set()
    
    # Iterate through every possible 'chomp' move
    rows = len(state)
    for r in range(rows):
        # In row r, we can bite at column c
        # Columns are 0-indexed. 
        # We can bite anywhere from 0 to state[r]-1.
        for c in range(state[r]):
            
            # Optimization: You cannot bite at (0,0) to survive. 
            # Biting (0,0) ends the game immediately (equivalent to moving to a terminal state).
            if r == 0 and c == 0:
                continue

            # Construct the new state after the chomp
            new_state_list = []
            for i in range(rows):
                if i < r:
                    # Rows above the bite are unaffected
                    new_state_list.append(state[i])
                else:
                    # Rows at or below the bite are limited by column c
                    new_val = min(state[i], c)
                    if new_val > 0:
                        new_state_list.append(new_val)
            
            # Convert to tuple for hashing
            new_state = tuple(new_state_list)
            
            # Handle the edge case where the board is wiped empty (someone ate 0,0)
            # Usually we treat the (1,) state as the effective end for calculation,
            # but if a move leads to empty, that move is fatal.
            if not new_state: 
                continue
                
            reachable_grundy_values.add(sprague_grundy(new_state))

    return mex(reachable_grundy_values)

# Calculate Sprague-Grundy values for all states up to (5,5,5)
all_pos = []
num = 5
for i in range(0, num + 1):
    for j in range(0, num + 1):
        for k in range(0, num + 1):
            if i >= j >= k:
                all_pos.append((i, j, k))

print(f"{'State':<15} | {'G-Value':<10} | {'Prediction'}")
print("-" * 40)

for pos in all_pos:
    g_val = sprague_grundy(pos)
    status = "LOSS" if g_val == 0 else "WIN"
    print(f"{str(pos):<15} | {g_val:<10} | {status}")

State           | G-Value    | Prediction
----------------------------------------
(0, 0, 0)       | 0          | LOSS
(1, 0, 0)       | 0          | LOSS
(1, 1, 0)       | 1          | WIN
(1, 1, 1)       | 2          | WIN
(2, 0, 0)       | 1          | WIN
(2, 1, 0)       | 0          | LOSS
(2, 1, 1)       | 3          | WIN
(2, 2, 0)       | 2          | WIN
(2, 2, 1)       | 0          | LOSS
(2, 2, 2)       | 4          | WIN
(3, 0, 0)       | 2          | WIN
(3, 1, 0)       | 3          | WIN
(3, 1, 1)       | 0          | LOSS
(3, 2, 0)       | 0          | LOSS
(3, 2, 1)       | 1          | WIN
(3, 2, 2)       | 3          | WIN
(3, 3, 0)       | 4          | WIN
(3, 3, 1)       | 3          | WIN
(3, 3, 2)       | 1          | WIN
(3, 3, 3)       | 5          | WIN
(4, 0, 0)       | 3          | WIN
(4, 1, 0)       | 2          | WIN
(4, 1, 1)       | 1          | WIN
(4, 2, 0)       | 4          | WIN
(4, 2, 1)       | 5          | WIN
(4, 2, 2)       | 0          | LOSS
